# Chapter 6: Precognition (逐步思考)

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## 设置

运行以下设置单元格以加载您的 API 密钥并建立 get_completion 辅助函数。

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## 课程
如果有人叫醒您并立即开始问您几个复杂的问题，您必须马上回答，您会怎么做？可能不会比给你时间 **先思考答案** 做得更好。
猜猜怎么着？LLM 也是一样的。
给 LLM 时间逐步思考有时会让 LLM 更准确，特别是对于复杂的任务。然而，只有**大声思考才算数**。您不能要求 LLM 思考但只输出答案 - 在这种情况下，实际上没有发生任何思考。

### 示例
在下面的提示词中，人类读者很容易看出第二句话与第一句话矛盾。但LLM 却过于字面地理解了“unrelated”这个词。

In [10]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

The sentiment of this movie review is positive.

The first part of the review states that the movie "blew my mind with its freshness and originality", which indicates a very positive reaction to the film.

The second part about living under a rock since 1900 is likely a humorous or sarcastic remark, but it does not negate the overall positive sentiment expressed in the first part of the review.

So based on the language used, this movie review has a positive sentiment, despite the somewhat tongue-in-cheek final statement.


为了改进 LLM 的回答，让我们允许 LLM 在回答之前先进行思考。我们可以通过明确写出 **LLM 需要采取的步骤来处理和思考任务**。再加上一点角色提示，这样可以让 LLM 更深入地理解这条评论。

In [11]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

<positive-argument>
- The review suggests the movie has a sense of freshness and originality, which can be seen as positive attributes.
- The reviewer's admission of living under a rock since 1900 implies that the movie is groundbreaking and innovative, something the reviewer has not experienced in a long time.
</positive-argument>

<negative-argument>
- The reviewer's sarcastic tone and the fact that they have been "living under a rock since 1900" suggests that the movie is not actually fresh or original, but rather something that has been done before.
- The reviewer's statement about living under a rock could be interpreted as a criticism of the movie, implying that it is not keeping up with the times or is behind the times.
</negative-argument>

Based on the arguments presented, the sentiment of the review is more negative than positive. The sarcastic tone and the implication that the movie is not as fresh or original as the reviewer claims suggest that the overall sentiment is nega

**LLM 有时对顺序很敏感**。这个例子处于 LLM 理解细微文本能力的前沿，当我们将前一个例子中的论点顺序交换，使负面论点在前、正面论点在后时，这会让 LLM 的整体判断变为正面。
在大多数情况下（但也不是全部，令人困惑的是），**LLM更有可能选择两个选项中的第二个**，这可能是因为在其来自网络的训练数据中，第二个选项更有可能是正确的。

In [12]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

<negative-argument>
The reviewer's statement about living under a rock since 1900 could be interpreted as a negative comment, suggesting that the movie's "freshness and originality" are not actually new or innovative, but rather something the reviewer has been unaware of for a very long time. This could imply that the movie is not as groundbreaking as the reviewer claims.
</negative-argument>

<positive-argument>
The reviewer's main statement about the movie "blowing their mind with its freshness and originality" is a very positive sentiment, indicating that the reviewer found the movie to be highly innovative and engaging. The fact that they were so impressed by the movie's qualities suggests a very positive overall review.
</positive-argument>

Based on the provided information, the review sentiment is more positive than negative. The reviewer's main statement about the movie being "fresh and original" and "blowing their mind" outweighs the potentially negative implication of living 


**让 LLM 进行思考可以让 LLM 的答案从错误变为正确**。在许多 LLM 出错的情况下，就是这么简单！
让我们通过一个 LLM 回答错误的例子，来看一下如何通过让LLM 思考来修正这个问题。

In [13]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

Here's a famous movie starring an actor born in 1956:

The Shawshank Redemption (1994) starring Tim Robbins. Tim Robbins was born on October 16, 1958.


让我们通过要求 Claude 逐步思考来修复这个问题，这次用 `<brainstorm>` 标签。

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))

Here's a brainstorm of some actors and their birth years:

<brainstorm>
- Tom Hanks (1956)
- Denzel Washington (1954)
- Julia Roberts (1967)
- Harrison Ford (1942)
- Meryl Streep (1949)
</brainstorm>

A famous movie starring an actor born in 1956 is:

Forrest Gump, starring Tom Hanks.


If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## 练习
- [Exercise 6.1 - Classifying Emails](#exercise-61---classifying-emails)
- [Exercise 6.2 - Email Classification Formatting](#exercise-62---email-classification-formatting)


### 练习 6.1 - 邮件分类
在本练习中，我们将指导 LLM 将邮件分为以下几类：
(A) 售前咨询
(B) 商品损坏或有缺陷
(C) 账单问题
(D) 其他（请说明）
练习的第一部分，请修改 PROMPT，**让 LLM 输出正确的分类**，并且只输出分类结果。您的答案需要 **包含正确选项的字母（A - D），带括号，以及类别名称**。
请参考 EMAILS 列表中每封邮件旁边的注释，了解每封邮件应该归入哪个类别。

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""



# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "A\\) P",
    "B": "B\\) B",
    "C": "C\\) B",
    "D": "D\\) O"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ If you want a hint, run the cell below!

### 提示：
本练习中的评分函数会查找正确的分类字母 + 右括号和类别名称的首字母，例如 "C) B" 或 "B) B" 等。
让我们一步一步完成这个练习：
1. Claude 如何知道你想用哪些分类？要告诉它！请直接在提示中写出你想要的四个分类。一定要包含带括号的字母，方便分类。可以使用 XML 标签来组织你的提示，并让 Claude 清楚地知道分类的开始和结束位置。
2. 尽量减少多余的文本，让 Claude 直接用分类结果作答，并且只输出分类结果。有多种方法可以做到这一点，比如“代表 Claude 发言”（从句子的开头到一个左括号，让 Claude 知道你希望答案以括号字母开头），或者直接告诉 Claude 你只要分类结果，跳过前言。如果需要复习这些技巧，可以参考第2章和第5章。
3. Claude 可能仍然会分类错误，或者在回答时没有包含类别名称。要解决这个问题，请明确告诉 Claude 在答案中包含完整的类别名称。
4. 请确保你的提示模板中仍然有 {email}，这样我们才能正确地将邮件内容替换进去，让 Claude 进行评估。

### Exercise 6.2 - Email Classification Formatting
In this exercise, we're going to refine the output of the above prompt to yield an answer formatted exactly how we want it. 

Use your favorite output formatting technique to make Claude wrap JUST the letter of the correct classification in `<answer></answer>` tags. For instance, the answer to the first email should contain the exact string `<answer>B</answer>`.

Refer to the comments beside each email in the `EMAILS` list if you forget which letter category is correct for each email.

### Exercise 6.2 - 邮件分类格式化
在本练习中，我们将完善上面提示的输出，使其答案格式完全符合我们的要求。
请使用你最喜欢的输出格式化技巧，让 Claude 只把正确分类的字母包裹在 <answer></answer> 标签中。例如，第一个邮件的答案应包含完全一致的字符串 <answer>B</answer>。
如果你忘记每封邮件对应哪个字母分类，请参考 EMAILS 列表中每封邮件旁边的注释。

In [ ]:


# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as (A) Pre-sale question, (B) Broken or defective item ,(C) Billing question , (D) Other (please explain): {email},only put full catalog  number in <answer> tags,skipping the preamble。 here is the example: <example> <answer>A</answer> <example>"""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ If you want a hint, run the cell below!

本练习中的评分函数只会查找被 <answer> 标签包裹的正确字母，例如“<answer>B</answer>”。正确的分类字母与上一个练习相同。
有时候，最简单的方法就是给 Claude 一个你希望其输出格式的示例。只要记得用 <example></example> 标签包裹你的示例！另外要注意，如果你为 Claude 的回复预填充了任何内容，Claude 实际上不会把这些内容作为它回复的一部分输出。

### 恭喜！

如果您已经完成了到目前为止的所有练习，您就可以进入下一章了。祝您提示词工程愉快